In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
from scipy.stats import norm, t
import scipy
import os
from functools import partial
from math import log, sqrt, exp, isclose
from scipy.optimize import fsolve
import seaborn as sns
import statsmodels.api as sm
import warnings
import inspect

In [ ]:
#initial conditions
current_date = datetime.strptime("10/30/2023", "%m/%d/%Y")  # 当前日期
rf = 0.0525  # 无风险利率
dy = 0.0057  # 股息收益率
current_price = 170.15  # 标的资产当前价格
#strike: 行权价格
#last price = market price

# 读取期权数据
options = pd.read_csv("FinTech545_Fall2024/Week06/Project/AAPL_Options.csv")

# 转换期权到期日期为 datetime 格式
options["Expiration"] = pd.to_datetime(options["Expiration"], format="%m/%d/%Y")

# TTM (Time to Maturity)
options["ttm"] = (options["Expiration"] - current_date).dt.days / 365

In [ ]:
def gbsm(call, current_price, strike, ttm, rf, dy, ivol):    """
    Black-Scholes-Merton 期权定价模型
    """
    b = rf - dy
    d1 = (np.log(current_price / strike) + (b + (ivol ** 2) / 2) * ttm) / (ivol * np.sqrt(ttm))
    d2 = d1 - ivol * np.sqrt(ttm)
    if call:
        return current_price * np.exp((b - rf) * ttm) * norm.cdf(d1) - strike * np.exp(-rf * ttm) * norm.cdf(d2)
    else:
        return strike * np.exp(-rf * ttm) * norm.cdf(-d2) - current_price * np.exp((b - rf) * ttm) * norm.cdf(-d1)

def find_iv(option_type, last_price, current_price, strike, ttm, rf, dy):
    """
    使用数值方法计算隐含波动率。
    """
    call = option_type == "Call"
    b = rf - dy
    def objective(ivol):
        return gbsm(call, current_price, strike, ttm, rf, dy, ivol) - last_price
    try:
        result = root_scalar(objective, bracket=[1e-6, 5.0], method="brentq")
        return result.root
    except ValueError:
        return np.nan


In [ ]:
# Black-Scholes-Merton 二叉树模型 (假设 bt_american 函数已定义)
def find_zero(call, current_s, strike, ttm, rf, div_amt, days_div, market_price):
    def objective(iv):
        return (
            bt_american(call, current_s, strike, ttm, rf, b, iv, int(ttm * mult * 365))
            - market_price
        )
    result = root_scalar(objective, bracket=[0.01, 3], method="brentq")
    return result.root

In [ ]:
def bt_american(call, underlying, strike, ttm, rf, b, ivol, N, div_time=None, div=None):
    """
    Binomial Tree for American Option Pricing with optional dividend consideration.
        float: Option price.
    """
    # Determine option type
    z = 1 if call else -1

    # Time step
    dt = ttm / N
    disc = np.exp(-rf * dt)

    # Up and down factors
    u = np.exp(ivol * np.sqrt(dt))
    d = 1 / u

    # Risk-neutral probabilities
    p = (np.exp(b * dt) - d) / (u - d)

    # Helper functions for node indexing
    def n_nodes(n):
        return (n + 1) * (n + 2) // 2

    def node_index(i, j):
        return n_nodes(j - 1) + i

    # Initialize the option values array
    C = np.empty(n_nodes(N), dtype=float)

    # If no dividends are provided, use standard binomial tree
    if div_time is None or div is None:
        for j in range(N, -1, -1):  # Backward induction
            for i in range(j, -1, -1):
                S = underlying * u ** i * d ** (j - i)
                idx = node_index(i, j)
                C[idx] = max(0, z * (S - strike))
                if j < N:
                    continuation = disc * (p * C[node_index(i + 1, j + 1)] +
                                           (1 - p) * C[node_index(i, j + 1)])
                    C[idx] = max(C[idx], continuation)
        return C[0]

    # If dividends are provided, adjust the tree
    new_ttm = ttm - div_time * dt
    new_N = N - div_time

    for j in range(div_time, -1, -1):
        for i in range(j, -1, -1):
            S = underlying * u ** i * d ** (j - i)
            idx = node_index(i, j)

            # Exercise value
            val_exe = max(0, z * (S - strike))

            # Continuation value
            if j < div_time:
                continuation = disc * (p * C[node_index(i + 1, j + 1)] +
                                       (1 - p) * C[node_index(i, j + 1)])
            else:
                # Recursively compute option value after dividend payment
                continuation = binomial_tree(
                    call, S - div, strike, new_ttm, rf, b, ivol, new_N, div_time=None, div=None
                )

            C[idx] = max(val_exe, continuation)

    return C[0]

#apply:
#无股息
#price = bt_american(call, underlying, strike, ttm, rf, b, ivol, N)
#有股息
#div_time = 10  # 股息支付时间步
#div = 2.0      # 股息金额
#price_with_div = bt_american(call, underlying, strike, ttm, rf, b, ivol, N, div_time=div_time, div=div)
#print(f"American Option Price (With Dividend): {price_with_div}")

In [ ]:
# 计算夏普比率的函数
def sharpe_ratio(w, mu, covar, rf):
    """
    计算投资组合的夏普比率。

    参数:
        w (numpy.ndarray): 投资组合权重向量。
        mu (numpy.ndarray): 每个资产的平均收益率。
        covar (numpy.ndarray): 资产收益率的协方差矩阵。
        rf (float): 无风险利率。

    返回:
        float: 夏普比率。
    """
    w = np.array(w)
    m = np.dot(w, mu) - rf  # 投资组合的超额收益
    s = np.sqrt(np.dot(np.dot(w, covar), w))  # 投资组合的波动率
    return m / s

In [ ]:
#calcluate greeks
def calculate_d1(S, X, T, implied_vol, b):
  return (np.log(S / X) + (b + implied_vol ** 2 / 2) * T) / (implied_vol * np.sqrt(T))

def calculate_d2(d1, T, implied_vol):
  return d1 - implied_vol * np.sqrt(T)

def gbsm_delta(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  delta = norm.cdf(d1 * is_call, 0, 1) * is_call
  return delta

def gbsm_gamma(option_type, S, X, T, implied_vol, r, b):
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  gamma = norm.pdf(d1, 0, 1) / (S * implied_vol * np.sqrt(T))
  return gamma

def gbsm_vega(option_type, S, X, T, implied_vol, r, b):
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  vega = S * norm.pdf(d1, 0, 1) * np.sqrt(T)
  return vega

def gbsm_theta(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  theta = -S * np.exp((b - r) * T) * norm.pdf(d1, 0, 1) * implied_vol / (2 * np.sqrt(T)) \
          -(b - r) * S * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call \
          -r * X * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
  return theta

def gbsm_rho(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  rho = X * T * np.exp(-r * T) * norm.cdf(d2 * is_call, 0, 1) * is_call
  return rho

def gbsm_carry_rho(option_type, S, X, T, implied_vol, r, b):
  is_call = 1 if option_type == "Call" else -1
  d1 = calculate_d1(S, X, T, implied_vol, b)
  d2 = calculate_d2(d1, T, implied_vol)
  carry_rho = S * T * np.exp((b - r) * T) * norm.cdf(d1 * is_call, 0, 1) * is_call
  return carry_rho

In [ ]:
import inspect

# calculate first order derivative
def first_order_der(func, x, delta):
  return (func(x + delta) - func(x - delta)) / (2 * delta)

# calculate second order derivative
def second_order_der(func, x, delta):
  return (func(x + delta) + func(x - delta) - 2 * func(x)) / delta ** 2

def cal_partial_derivative(func, order, arg_name, delta=1e-3):
  # initialize for argument names and order
  arg_names = list(inspect.signature(func).parameters.keys())
  derivative_fs = {1: first_order_der, 2: second_order_der}

  def partial_derivative(*args, **kwargs):
    # parse argument names and order
    args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
    arg_val = args_dict.pop(arg_name)

    def partial_f(x):
      p_kwargs = {arg_name:x, **args_dict}
      return func(**p_kwargs)
    return derivative_fs[order](partial_f, arg_val, delta)
  return partial_derivative

In [ ]:
# Delta 计算函数
def bt_delta(call, underlying, strike, ttm, rf, div_amt, days_div, ivol, N):
    def price_func(underlying_price):
        return bt_american(call, underlying_price, strike, ttm, rf, b, ivol, N)
    # Change: Add a check to ensure np.gradient returns a valid result
    gradient_result = np.gradient([price_func(underlying + 1e-6), price_func(underlying - 1e-6)], [1e-6, -1e-6])
    if gradient_result.size > 0:
        return gradient_result[0]
    else:
        return 0.0